##AtmaBodh System

In [ ]:
# !pip install langchain langchain_openai langchain_community langchain_core sentence-transformers faiss-cpu openai pandas > /dev/null

###Import Dataset

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "bhagavad_gita_verses.csv"

bg_data = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "madhurpant/bhagavad-gita-verses-dataset",
  file_path,
)

bg_data.columns

Using Colab cache for faster access to the 'bhagavad-gita-verses-dataset' dataset.


Index(['chapter_number', 'chapter_title', 'chapter_verse', 'translation'], dtype='object')

###Import Required Libraries

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.document_loaders import DataFrameLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_classic.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_community.vectorstores import FAISS
from langchain_classic.chains import RetrievalQA
from google.colab import userdata
import faiss
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

###Core Variables

In [ ]:
loader = DataFrameLoader(bg_data, page_content_column="translation")
docs = loader.load()

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
vectordb = FAISS.from_documents(docs, embeddings)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.4)

###RAG Chain

In [ ]:
prompt = ChatPromptTemplate.from_template("""
You are Vaani, an expert on the Bhagavad Gita.
Use the conversation history and the context below.
Answer ONLY using the context.
Always mention Chapter and Verse (Shlok) and the lines of shloks, in the place where the context from a shlok is explained.
Be polite, creative in your words, don't repeat same opening or closing lines again and again.
Also sound interested in the conversation and intellectual.

Conversation History:
{history}

Context:
{context}

Question:
{input}

Answer (with reference):
""")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {
        "context": (RunnableLambda(lambda x: x["input"]) | retriever | RunnableLambda(format_docs)),
        "input": RunnableLambda(lambda x: x["input"]),
        "history": RunnableLambda(lambda x: x.get("history", "")),
    }
    | prompt
    | llm
)

* Defining ChatBot

In [ ]:
store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

chatbot = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

###Working Chatbot (Abstracted)

In [ ]:
print("""🙏 ĀtmaBodh Conversation System :
        Hi, I am Vaani.❤️
        Feel free to ask any question and explore thoughtful and spiritual answers.
        Type 'HareKrishna' anytime to end the chat.\n""")


while True:
    user_input = input("You: ")
    if user_input.lower() == "harekrishna":
        break
    response = chatbot.invoke(
        {"input": user_input},
        config={"configurable": {"session_id": "gita_chat"}}
    )

    print("\nVaani:", response.content, "\n")

🙏 ĀtmaBodh Conversation System :
        Hi, I am Vaani.❤️
        Feel free to ask any question and explore thoughtful and spiritual answers.
        Type 'HareKrishna' anytime to end the chat.

You: what is dharma?

Vaani: Dharma, as articulated in the Bhagavad Gita, encompasses the moral and ethical duties that align with an individual's nature and societal role. This concept is vividly expressed in Chapter 3, Verse 35 (3.35), which states: "It is better to do one’s own dharma, even though imperfectly, than to do another’s dharma, even though perfectly. By doing one’s innate duties, a person does not incur sin." This verse underscores the significance of adhering to one's own duties, as they resonate with one's true self, rather than attempting to fulfill the roles of others, which can lead to disarray and misalignment with one's inherent qualities.

Dharma serves as a guiding principle that not only fosters personal integrity but also contributes to the overall harmony and righteou